In [ ]:
# ###########################reference link - Fuzzy company name match ###################################

# https://stackoverflow.com/questions/6400416/figure-out-if-a-business-name-is-very-similar-to-another-one-python

# http://nadbordrozd.github.io/blog/2016/07/29/datamatching-part-3-match-scoring/
# https://blog.ouseful.info/2017/09/04/simple-text-analysis-using-python-identifying-named-entities-tagging-fuzzy-string-matching-and-topic-modelling/
# https://bergvca.github.io/2017/10/14/super-fast-string-matching.html
################## Procedures#############################################################################

#### 1. name cleaning /standardization 
# lowercase name
# remove special symbols 
# remove numbers  if nessary 

##### 2. fuzzy name match 
# remove stop words  before match ( the , LLC etc ), split text into words(word tokenize, tf-idf, n-gram )
# apply match algorithms ()

## first try in python then try in sql 

In [28]:
# import lib 
import os
import pandas as pd

In [29]:
# read data in 
df=pd.read_excel('fuzzy_name.xlsx')

In [30]:
 df.head(5)

,InfoUSA_name,infoUSA_adr,CS_name,CS_address,str_adr_apt,MD_NAME,MD_Address
0,VESTA PROPERTY SVC,245 RIVERSIDEAVE,"ARTESIA CLUB HOLDING , LLC C/O VESA PROPERTY S...",245 RIVERSIDE AVE STE 250,245 Riverside Ave,"Artesia Club Holding, LLC c/o VESA Property Se...",245 Riverside Ave
1,BAYSIDE PHYSICAL THRPY CHIRO,21315 33RDRD,"BAYSIDE PT, CHIRO, ACU",213-15 33RD ST,21315 33rd Rd,"Bayside PT, Chiro, ACU",21315 33rd Rd
2,BACKHOUSE RESTAURANT,716 MONTEREY PASSRD,"BACKHOUSE HUNTINGTON, LLC DBA THE BACKHOUSE",716 MONTEREY PASS RD,716 Monterey Pass Rd,"Backhouse Huntington, LLC DBA the Backhouse",716 Monterey Pass Rd
3,BARBACOA MEXICAN GRILL CORP,1335S 700 W,"BARBACOA, INC DBA BARBACOA MEXICAN GRILL",1335 S 700 W,1335 S 700 W,"Barbacoa, Inc. DBA Barbacoa Mexican Grill",1335 S 700 W
4,MC DONALD'S,25297 LANKFORDHWY,"BAXTER ENTERPRISES, LLC/ MCDONALDS",25297 LANKFORD HWY,25297 Lankford Hwy,"Baxter Enterprises, LLC/ McDonalds",25297 Lankford Hwy


In [31]:
df.dtypes

InfoUSA_name    object
infoUSA_adr     object
CS_name         object
CS_address      object
str_adr_apt     object
MD_NAME         object
MD_Address      object
dtype: object

In [32]:
# Part1 name standardiztion 
# metho1 :
# https://medium.com/@chaimgluck1/have-messy-text-data-clean-it-with-simple-lambda-functions-645918fcc2fc
# step1 lowercase string  
# step2 remove punctuation string 
# step3 remove digit from string 
import string 
def name_clean(df,name):
    df.name = df.name.apply(lambda x: x.lower())
    df.name = df.name.apply(lambda x: x.translate(None, string.punctuation))
    df.name = df.name.apply(lambda x: x.translate(None, string.digits)) # remove numbers 

In [33]:
# Method2 : standazied name
import string 
from cleanco import cleanco
# step1 Remove all punctuation:
# step2 clean LLC, Corp , ASSOC etc 
def remove_name(df,x,x2):
     # clean name , remove special symbol and lowcase the char
    #df[x2] = [i.strip().replace('.','').replace('''''', '').replace('-', '').replace(',','')for i in df[x].str.lower()]
    df[x2] =df[x].str.replace('[{}]'.format(string.punctuation), '').str.lower()
    # remove digit
    df[x2] =df[x2].str.replace('[{}]'.format(string.digits), '')
    # remove LLC, corp ,INC from company name,etc
    df[x2]= df[x2].apply(cleanco,1)
    df[x2]= df[x2].apply(lambda x: x.clean_name())
    return df

In [34]:
# apply function to the dataframe 
df2=remove_name(df,'CS_name','CS_name_clean')
df2=remove_name(df,'InfoUSA_name','InfoUSA_name_clean')
df2=remove_name(df,'MD_NAME','MD_name_clean')

In [35]:
df2.head(5)
    

,InfoUSA_name,infoUSA_adr,CS_name,CS_address,str_adr_apt,MD_NAME,MD_Address,CS_name_clean,InfoUSA_name_clean,MD_name_clean
0,VESTA PROPERTY SVC,245 RIVERSIDEAVE,"ARTESIA CLUB HOLDING , LLC C/O VESA PROPERTY S...",245 RIVERSIDE AVE STE 250,245 Riverside Ave,"Artesia Club Holding, LLC c/o VESA Property Se...",245 Riverside Ave,"ARTESIA CLUB HOLDING , LLC C/O VESA PROPERTY S...",VESTA PROPERTY SVC,"Artesia Club Holding, LLC c/o VESA Property Se..."
1,BAYSIDE PHYSICAL THRPY CHIRO,21315 33RDRD,"BAYSIDE PT, CHIRO, ACU",213-15 33RD ST,21315 33rd Rd,"Bayside PT, Chiro, ACU",21315 33rd Rd,"BAYSIDE PT, CHIRO, ACU",BAYSIDE PHYSICAL THRPY CHIRO,"Bayside PT, Chiro, ACU"
2,BACKHOUSE RESTAURANT,716 MONTEREY PASSRD,"BACKHOUSE HUNTINGTON, LLC DBA THE BACKHOUSE",716 MONTEREY PASS RD,716 Monterey Pass Rd,"Backhouse Huntington, LLC DBA the Backhouse",716 Monterey Pass Rd,"BACKHOUSE HUNTINGTON, LLC DBA THE BACKHOUSE",BACKHOUSE RESTAURANT,"Backhouse Huntington, LLC DBA the Backhouse"
3,BARBACOA MEXICAN GRILL CORP,1335S 700 W,"BARBACOA, INC DBA BARBACOA MEXICAN GRILL",1335 S 700 W,1335 S 700 W,"Barbacoa, Inc. DBA Barbacoa Mexican Grill",1335 S 700 W,"BARBACOA, INC DBA BARBACOA MEXICAN GRILL",BARBACOA MEXICAN GRILL,"Barbacoa, Inc. DBA Barbacoa Mexican Grill"
4,MC DONALD'S,25297 LANKFORDHWY,"BAXTER ENTERPRISES, LLC/ MCDONALDS",25297 LANKFORD HWY,25297 Lankford Hwy,"Baxter Enterprises, LLC/ McDonalds",25297 Lankford Hwy,"BAXTER ENTERPRISES, LLC/ MCDONALDS",MC DONALD'S,"Baxter Enterprises, LLC/ McDonalds"


In [36]:
# Part 2 fuzzy match prepare
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhula\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
#step1 remove stop words
# https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe
from nltk.corpus import stopwords
df2['CS_name_clean2'] = df2['CS_name_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))
df2['InfoUSA_name_clean2'] = df2['InfoUSA_name_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))

In [39]:
df2[['InfoUSA_name_tk','CS_name_tk','InfoUSA_name_clean2','CS_name_clean2','InfoUSA_name','CS_name']].tail(5)

,InfoUSA_name_tk,CS_name_tk,InfoUSA_name_clean2,CS_name_clean2,InfoUSA_name,CS_name
137,"[BINSKY, &, SNYDER, MECHANICAL]","[BINSKY, &, SNYDER]",BINSKY & SNYDER MECHANICAL,BINSKY & SNYDER,BINSKY & SNYDER MECHANICAL,"BINSKY & SNYDER, LLC"
138,"[BINSKY, HOME]","[BINSKY, &, SNYDER]",BINSKY HOME,BINSKY & SNYDER,BINSKY HOME,"BINSKY & SNYDER, LLC"
139,"[BJ, 'S]","[BJS, RESTAURANTS]",BJ'S,BJS RESTAURANTS,BJ'S 508,"BJS RESTAURANTS, INC."
140,"[FINAL, TOUCH, COLLISION, CTR]","[FINAL, TOUCH, COLLISION, CENTER]",FINAL TOUCH COLLISION CTR,FINAL TOUCH COLLISION CENTER,FINAL TOUCH COLLISION CTR,FINAL TOUCH COLLISION CENTER
141,"[PREFERRED, CARE, PC]","[NEW, SENSE, ACUPUNCTURE]",PREFERRED CARE PC,NEW SENSE ACUPUNCTURE,PREFERRED CARE PC,NEW SENSE ACUPUNCTURE


In [40]:
df2[['InfoUSA_name_tk','CS_name_tk','InfoUSA_name_clean2','CS_name_clean2','InfoUSA_name','CS_name']].head(5)

,InfoUSA_name_tk,CS_name_tk,InfoUSA_name_clean2,CS_name_clean2,InfoUSA_name,CS_name
0,"[VESTA, PROPERTY, SVC]","[ARTESIA, CLUB, HOLDING, ,, LLC, C/O, VESA, PR...",VESTA PROPERTY SVC,"ARTESIA CLUB HOLDING , LLC C/O VESA PROPERTY S...",VESTA PROPERTY SVC,"ARTESIA CLUB HOLDING , LLC C/O VESA PROPERTY S..."
1,"[BAYSIDE, PHYSICAL, THRPY, CHIRO]","[BAYSIDE, PT, ,, CHIRO, ,, ACU]",BAYSIDE PHYSICAL THRPY CHIRO,"BAYSIDE PT, CHIRO, ACU",BAYSIDE PHYSICAL THRPY CHIRO,"BAYSIDE PT, CHIRO, ACU"
2,"[BACKHOUSE, RESTAURANT]","[BACKHOUSE, HUNTINGTON, ,, LLC, DBA, THE, BACK...",BACKHOUSE RESTAURANT,"BACKHOUSE HUNTINGTON, LLC DBA THE BACKHOUSE",BACKHOUSE RESTAURANT,"BACKHOUSE HUNTINGTON, LLC DBA THE BACKHOUSE"
3,"[BARBACOA, MEXICAN, GRILL]","[BARBACOA, ,, INC, DBA, BARBACOA, MEXICAN, GRILL]",BARBACOA MEXICAN GRILL,"BARBACOA, INC DBA BARBACOA MEXICAN GRILL",BARBACOA MEXICAN GRILL CORP,"BARBACOA, INC DBA BARBACOA MEXICAN GRILL"
4,"[MC, DONALD, 'S]","[BAXTER, ENTERPRISES, ,, LLC/, MCDONALDS]",MC DONALD'S,"BAXTER ENTERPRISES, LLC/ MCDONALDS",MC DONALD'S,"BAXTER ENTERPRISES, LLC/ MCDONALDS"


In [41]:
# step2 -approach2 find the keywords in the company name , i.e. stop words 
from collections import Counter

def keywords(df,name):
    key = df[name].unique()
    names_freq = Counter()
    for x in key:
        names_freq.update(str(x).split(" "))
    key_words = [word for (word,_) in names_freq.most_common(30)]
    return key_words


In [42]:
listkey=keywords(df2,'CS_name_clean') + list(set(keywords(df2,'InfoUSA_name_clean')) - set(keywords(df2,'CS_name_clean')))
print (listkey)

['&', 'HOLDINGS', 'A', 'LLC', 'DBA', 'ADVANCED', 'CO', 'SERVICES', 'INC', ',', '-', 'AUTO', 'GROUP', 'OF', 'SYSTEMS', 'PROSTHETICS', 'AMERICAN', 'PROPERTIES', 'B', 'BACKHOUSE', 'THE', 'AND', 'MAINTENANCE', 'PLUS', 'INVESTMENT', 'FAMILY', 'ACG', 'TEXAS', 'ACTION', 'RESORT', 'BBL', 'DENTAL', 'CARE', 'ON', 'MANAGEMENT', 'AERO', 'SNYDER', 'SOUTH', 'PIZZA', 'ACC', 'BINSKY', 'USA', 'INVESTMENTS', 'SVC', 'ATLANTIC', 'PROPERTY']


In [ ]:
## step3 Matching pair names 
# algorithm1 -fuzzywuzzy
# use fuzzywuzzy.token_sort_ratio to matching the names. Different form the basic fuzzywuzzy.ratio which use Levenshtein Distance to calculate the differences, it allow the token (words) in a name to swap order and still give a 'perfect' match. (ref: https://github.com/seatgeek/fuzzywuzzy)
# http://jonathansoma.com/lede/algorithms-2017/classes/fuzziness-matplotlib/fuzzing-matching-in-pandas-with-fuzzywuzzy/
# https://stackoverflow.com/questions/53103848/fuzzy-matching-two-columns-in-the-same-dataframe-using-python
# fuzz.token_sort_ratio ignores word order
# fuzz.token_sort_ratio orders all of the words first, so “KENNEDY JOHN” and “JOHN KENNEDY” would be the same.

In [43]:
#pip3 install fuzzywuzzy[speedup]
# conda install -c conda-forge fuzzywuzzy 
from fuzzywuzzy import process, fuzz


In [44]:
#df2['fuzz_score'] = fuzz.token_sort_ratio(df2['InfoUSA_name_tk'],df2['CS_name_tk'])
df2['fuzz_score']=df2.apply(lambda s: fuzz.partial_token_sort_ratio(s['CS_name_clean2'], s['InfoUSA_name_clean2']), axis=1)

In [45]:
df2[['InfoUSA_name_tk','CS_name_tk','InfoUSA_name','CS_name','fuzz_score']].tail(10)

,InfoUSA_name_tk,CS_name_tk,InfoUSA_name,CS_name,fuzz_score
132,"[BINGHAMTON, GIANT, MARKETS]","[BINGHAMTON, GIANT, MARKET]",BINGHAMTON GIANT MARKETS,"BINGHAMTON GIANT MARKET, INC.",100
133,"[BINSKY, &, SNYDER]","[BINSKY, &, SNYDER]",BINSKY & SNYDER,"BINSKY & SNYDER, LLC",100
134,"[BINSKY, &, SNYDER, HOME, SVC]","[BINSKY, &, SNYDER]",BINSKY & SNYDER HOME SVC LLC,"BINSKY & SNYDER, LLC",62
135,"[BINSKY, &, SNYDER]","[BINSKY, &, SNYDER]",BINSKY & SNYDER LLC,"BINSKY & SNYDER, LLC",100
136,"[BINSKY, &, SNYDER]","[BINSKY, &, SNYDER]",BINSKY & SNYDER LLC,"BINSKY & SNYDER, LLC",100
137,"[BINSKY, &, SNYDER, MECHANICAL]","[BINSKY, &, SNYDER]",BINSKY & SNYDER MECHANICAL,"BINSKY & SNYDER, LLC",62
138,"[BINSKY, HOME]","[BINSKY, &, SNYDER]",BINSKY HOME,"BINSKY & SNYDER, LLC",64
139,"[BJ, 'S]","[BJS, RESTAURANTS]",BJ'S 508,"BJS RESTAURANTS, INC.",75
140,"[FINAL, TOUCH, COLLISION, CTR]","[FINAL, TOUCH, COLLISION, CENTER]",FINAL TOUCH COLLISION CTR,FINAL TOUCH COLLISION CENTER,91
141,"[PREFERRED, CARE, PC]","[NEW, SENSE, ACUPUNCTURE]",PREFERRED CARE PC,NEW SENSE ACUPUNCTURE,41


In [47]:
# output results
writer = pd.ExcelWriter('match_1.xlsx', engine='xlsxwriter')
df2.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
# algorithm2 

In [38]:
# step2 split text into words 
#tokens = nltk.word_tokenize(name)  # ['THE', 'big', 'Pharma']
df2['CS_name_tk'] = df2['CS_name_clean2'].apply(nltk.word_tokenize) 
df2['InfoUSA_name_tk'] = df2['InfoUSA_name_clean2'].apply(nltk.word_tokenize) 